In [66]:
# load modules (install floris)
from floris.floris import Floris
import numpy as np
import matplotlib.pyplot as plt
import scipy
import gpflow
import GPy
import random
import time

from pygmo import hypervolume

# Visualization
from copy import deepcopy
from visualization_manager import VisualizationManager
from pareto import Pareto
from scipy.optimize import minimize
from scipy.stats import norm
from pareto import Pareto
%matplotlib inline

In [67]:
def windFarmPower(floris,wd, ws, yawAngle,scale):
    
    #set up wind direction and speed
    floris.farm.flow_field.wind_direction = np.radians(wd - 270) # frame of reference is west
    floris.farm.flow_field.wind_speed = ws
    floris.farm.flow_field.initial_flowfield = floris.farm.flow_field._initial_flowfield()
    floris.farm.flow_field.u_field = floris.farm.flow_field._initial_flowfield()
    
    
    turbines    = [turbine for _, turbine in floris.farm.flow_field.turbine_map.items()]
    for k,turbine in enumerate(turbines):
        turbine.yaw_angle = np.radians(yawAngle[k])
    floris.farm.flow_field.calculate_wake()
    
    power = np.zeros([len(yawAngle),1])
    totalPower = 0.0
    for i, turbine in enumerate(turbines):
        power[i]=turbine.power
        totalPower = totalPower + turbine.power    
    
    return power/scale, totalPower/scale/len(turbines)

In [68]:
#run wind farm configuration input with a single wind turbine
floris = Floris("example_input_single.json")
numWT = 1
scale = 1.0
#conventional default input is yawAngle = 0 degree
yawAngle0 = np.zeros(numWT)

#compute the wind turbine power vector and total wind farm power (for single wind turbine they are the same)
powerSingle,totalPower = windFarmPower(floris,0, 8, yawAngle0,scale)

for coord, turbine in floris.farm.turbine_map.items():
    print(str(coord) + ":")
    print("\tCp -", turbine.Cp)
    print("\tCt -", turbine.Ct)
    print("\tpower -", turbine.power)
    print("\tai -", turbine.aI)
    print("\taverage velocity -", turbine.get_average_velocity())

(0.0, 0.0):
	Cp - 0.46328782548262326
	Ct - 0.7661304442831962
	power - 1712005.1679717556
	ai - 0.2581996920407235
	average velocity - 7.85065163365446


In [69]:
floris = Floris("example_input_double.json")

In [70]:
wd=180;
ws=8;
num_tur = len(floris.farm.flow_field.turbine_map.items())
yawAngle = np.ones(num_tur)*0.01
power, totalPower = windFarmPower(floris, wd, 8, yawAngle, powerSingle)
totalPower

array([[0.99999909]])

In [71]:
def hypervolume_poi(Xcand, gp_models, pareto, reference, outdim, wind_dir):
    Xcand = np.atleast_2d(Xcand)
    Xcand = np.hstack((Xcand, np.ones((len(Xcand), 1)) * wind_dir))
    num_cells = pareto.bounds.lb.shape[0]
    N = Xcand.shape[0]

    # Extended Pareto front
    pf_ext = np.concatenate([-np.inf * np.ones([1, outdim], dtype=float), pareto.front, reference], 0)

    # Predictions for candidates, concatenate columns
    preds = [m.predict(Xcand) for m in gp_models]
    candidate_mean, candidate_var = (np.concatenate(moment, 1) for moment in zip(*preds))
    candidate_var = np.maximum(candidate_var, 1e-6)  # avoid zeros

    # Calculate the cdf's for all candidates for every predictive distribution in the data points
    normal = scipy.stats.norm(candidate_mean, np.sqrt(candidate_var))
    Phi = np.transpose(normal.cdf(np.expand_dims(pf_ext, 1)), [1, 0, 2])  # N x pf_ext_size x outdim
    # tf.gather_nd indices for bound points
    col_idx = np.tile(range(outdim), (num_cells,))
    ub_idx = np.stack((np.reshape(pareto.bounds.ub, [-1]), col_idx), axis=1).astype(int)  # (num_cells*outdim x 2)
    lb_idx = np.stack((np.reshape(pareto.bounds.lb, [-1]), col_idx), axis=1).astype(int)  # (num_cells*outdim x 2)
    
    # Calculate PoI
    P1 = np.zeros((N, num_cells*outdim))
    P2 = np.zeros((N, num_cells*outdim))
    for i in range(len(ub_idx)):
        for k in range(N):
            P1[k,i] = np.transpose(Phi, [1, 2, 0])[ub_idx[i][0],ub_idx[i][1], k]  # N x num_cell*outdim
            P2[k,i] = np.transpose(Phi, [1, 2, 0])[lb_idx[i][0],lb_idx[i][1], k]  # N x num_cell*outdim
    P = np.reshape(P1 - P2, [N, num_cells, outdim])
    PoI = np.sum(np.prod(P, axis=2), axis=1, keepdims=True)  # N x 1

    # Calculate Hypervolume contribution of points Y
    ub_points = np.zeros((1, num_cells*outdim))
    lb_points = np.zeros((1, num_cells*outdim))
    for i in range(len(ub_idx)):
        ub_points[0,i] = pf_ext[ub_idx[i][0],ub_idx[i][1]]
        lb_points[0,i] = pf_ext[lb_idx[i][0],lb_idx[i][1]]
    ub_points = np.reshape(ub_points, [num_cells, outdim])
    lb_points = np.reshape(lb_points, [num_cells, outdim])

    splus_valid = np.all(np.tile(np.expand_dims(ub_points, 1), [1, N, 1]) > candidate_mean,
                                axis=2)  # num_cells x N
    splus_idx = np.expand_dims(splus_valid.astype(np.float64), -1)  # num_cells x N x 1
    splus_lb = np.tile(np.expand_dims(lb_points, 1), [1, N, 1])  # num_cells x N x outdim
    splus_lb = np.maximum(splus_lb, candidate_mean)  # num_cells x N x outdim
    splus_ub = np.tile(np.expand_dims(ub_points, 1), [1, N, 1])  # num_cells x N x outdim
    splus = np.concatenate([splus_idx, splus_ub - splus_lb], axis=2)  # num_cells x N x (outdim+1)
    Hv = np.transpose(np.sum(np.prod(splus, axis=2), axis=0, keepdims=True))  # N x 1
    
    # return HvPoI
    return -np.multiply(Hv, PoI)

In [72]:
def sample_next_point(acquisition, gp_model, bounds, pareto, reference, outdim, wind_dir, N_mc = 5000):
    """
    acquisition : acquisition function of Gaussian processes
    gp_model : gpflow Gaussian process model
    eval_y : evaluated y list in current state
    bounds : boundary of next point
    n_restarts : number of restarts for scipy.minimize
    
    return : next x    
    """
    best_x = None
    best_acquisition_value = 0
    n_params = bounds.shape[0]
    points = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(N_mc, n_params))
    evaluations = acquisition(points, gp_model, pareto, reference, outdim, wind_dir)
    idx_best = np.argmin(evaluations, axis=0)
    
    result = minimize(fun=acquisition, x0=points[idx_best, :], bounds=bounds, method='L-BFGS-B',args=(gp_model, pareto, reference, outdim, wind_dir))
    if result.fun <= best_acquisition_value:
        best_acquisition_value = result.fun
        best_x = result.x

    return best_x

In [73]:
def multiobj_f(yawangle):
    Y1 = np.zeros(len(yawangle))[:,None]
    Y2 = np.zeros(len(yawangle))[:,None]
    for i,angle in enumerate(yawangle):
        power,totalPower = windFarmPower(floris, angle[-1], 8, angle[:-1], powerSingle)
        Y1[i] = -totalPower
        Y2[i] = np.sum(np.square(np.radians(angle[:-1]))) / np.square(np.radians(25.0)) / num_tur
    Y = np.hstack((Y1,Y2))
    return Y

In [74]:
minimum_yaw_angle = 0.0
maximum_yaw_angle = 25.0

bounds = np.zeros((num_tur, 2))
for i in range(num_tur):
    bounds[i,:] = [minimum_yaw_angle, maximum_yaw_angle]
    
# number of turbines + wind context
n_params = bounds.shape[0]+1

In [75]:
# initial data point of wind-farm
init_sample = 1
wd_init = np.random.uniform(90, 120, size=(init_sample, 1))

X = np.zeros((init_sample, bounds.shape[0]))
for i, yawangles in enumerate(np.random.uniform(bounds[:, 0], bounds[:, 1], (init_sample, bounds.shape[0]))):
    X[i,:] = yawangles
X = np.hstack((X, wd_init))
Y = multiobj_f(X)

In [76]:
num_output = Y.shape[1]
num_iter = 8
wd_context = np.random.uniform(90, 120, size=(num_iter, 1))
t1 = time.time()
    
for i in range(num_iter):
    print(i)
    pareto = Pareto(np.empty((0, num_output)))
    reference = np.ones((1, num_output))

    if i == 0:
        gp_models = [GPy.models.GPRegression(X.copy(), Y[:,[i]].copy(), kernel= GPy.kern.Matern52(input_dim=n_params, ARD=True)) for i in range(Y.shape[1])]
    elif i%5 != 0:
        gp_models = [GPy.models.GPRegression(X.copy(), Y[:,[i]].copy(), kernel= gp_models[i].kern, noise_var = gp_models[i].likelihood[0] ) for i in range(Y.shape[1])]
    else:
        gp_models = [GPy.models.GPRegression(X.copy(), Y[:,[i]].copy(), kernel= gp_models[i].kern, noise_var = gp_models[i].likelihood[0] ) for i in range(Y.shape[1])]
        for m in gp_models:
            m.optimize()
    
    trusted_X = np.array([X[j] for j in range(len(X)) if wd_context[i]-30 <= X[j,-1] <= wd_context[i]+30])
    context_X = np.hstack((trusted_X[:,:-1], np.ones((len(trusted_X), 1)) * wd_context[i]))    
    preds =  [m.predict(context_X) for m in gp_models]
    context_Y, var = (np.concatenate(moment, 1) for moment in zip(*preds))
    pareto.update(context_Y)
    pf = pareto.front
    f = np.max(pf, axis=0, keepdims=True) - np.min(pf, axis=0, keepdims=True)
    reference = np.max(pf, axis=0, keepdims=True) + 2 * f / pf.shape[0]
       
    next_point = np.atleast_2d(sample_next_point(hypervolume_poi, gp_models, bounds, pareto, reference, num_output, wd_context[i]))
    next_point = np.hstack((next_point, np.ones((1,1)) * wd_context[i]))
    X = np.append(X, next_point, axis = 0)
    function_value = multiobj_f(np.atleast_2d(next_point))
    Y = np.append(Y, function_value, axis = 0)
t2 = time.time()
print(t2-t1)

0
1
2
3
4
5


 /home/cwj/venv/local/lib/python3.5/site-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /home/cwj/venv/local/lib/python3.5/site-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add


6
7
1.6493444442749023


In [64]:
extended = np.tile(pareto.Y, (pareto.Y.shape[0], 1, 1))
dominance = np.sum(np.logical_and(np.all(extended <= np.swapaxes(extended, 0, 1), axis=2),
                                  np.any(extended < np.swapaxes(extended, 0, 1), axis=2)), axis=1)

In [67]:
pareto.Y[dominance == 0], dominance

(array([[-7.73379789e-01,  4.24653149e-01],
        [-7.73809290e-01,  4.44670241e-01],
        [-6.89312303e-01, -2.09266489e-07],
        [-7.40432153e-01,  2.26048466e-01]]), array([2, 0, 4, 0, 2, 3, 0, 0]))

In [70]:
np.argsort(pareto.front, axis=0)

array([[0, 3],
       [1, 2],
       [2, 1],
       [3, 0]])

In [73]:
outdim = 2
pf_idx = np.argsort(pareto.front, axis=0)
pf_ext_idx = np.vstack((np.zeros(outdim, dtype=np.int32), pf_idx + 1,
                        np.ones(outdim, dtype=np.int32) * pareto.front.shape[0] + 1))
pf_ext_idx

array([[0, 0],
       [1, 4],
       [2, 3],
       [3, 2],
       [4, 1],
       [5, 5]])

In [77]:
for i in range(pf_ext_idx[-1, 0]):
    print(i, 0)
    print(i+1, pf_ext_idx[-i-1, 1])
    print('')

0 0
1 5

1 0
2 1

2 0
3 2

3 0
4 3

4 0
5 4



In [79]:
np.max(pareto.front, axis=0, keepdims=True) + 2 * f / pareto.front.shape[0]

array([[-0.64706381,  0.66700547]])

In [81]:
pf = pareto.front
reference = np.max(pf, axis=0, keepdims=True) + 2 * f / pf.shape[0]

In [82]:
Xcand = next_point
num_cells = pareto.bounds.lb.shape[0]
N = Xcand.shape[0]

# Extended Pareto front
pf_ext = np.concatenate([-np.inf * np.ones([1, outdim], dtype=float), pareto.front, reference], 0)

# Predictions for candidates, concatenate columns
preds = [m.predict(Xcand) for m in gp_models]
candidate_mean, candidate_var = (np.concatenate(moment, 1) for moment in zip(*preds))
candidate_var = np.maximum(candidate_var, 1e-6)  # avoid zeros

# Calculate the cdf's for all candidates for every predictive distribution in the data points
normal = scipy.stats.norm(candidate_mean, np.sqrt(candidate_var))
Phi = np.transpose(normal.cdf(np.expand_dims(pf_ext, 1)), [1, 0, 2])  # N x pf_ext_size x outdim

In [87]:
np.round(pf_ext,3)

array([[  -inf,   -inf],
       [-0.774,  0.445],
       [-0.773,  0.425],
       [-0.74 ,  0.226],
       [-0.689, -0.   ],
       [-0.647,  0.667]])

In [105]:
function_value

array([[-0.75636357,  0.29728938]])

In [85]:
normal.cdf(np.expand_dims(pf_ext, 1))

array([[[0.00000000e+000, 0.00000000e+000]],

       [[3.53040542e-002, 1.00000000e+000]],

       [[4.75501116e-002, 1.00000000e+000]],

       [[1.00000000e+000, 2.37340304e-013]],

       [[1.00000000e+000, 9.89055387e-222]],

       [[1.00000000e+000, 1.00000000e+000]]])

In [118]:
Phi[0][2] = 0.04, 0.98
Phi[0][3] = 0.98, 0.02
Phi[0][4] = 0.99, 0.01

In [92]:
col_idx = np.tile(range(outdim), (num_cells,))
ub_idx = np.stack((np.reshape(pareto.bounds.ub, [-1]), col_idx), axis=1).astype(int)  # (num_cells*outdim x 2)
lb_idx = np.stack((np.reshape(pareto.bounds.lb, [-1]), col_idx), axis=1).astype(int)  # (num_cells*outdim x 2)
print(ub_idx)
print(lb_idx)

[[1 0]
 [5 1]
 [2 0]
 [1 1]
 [3 0]
 [2 1]
 [4 0]
 [3 1]
 [5 0]
 [4 1]]
[[0 0]
 [0 1]
 [1 0]
 [0 1]
 [2 0]
 [0 1]
 [3 0]
 [0 1]
 [4 0]
 [0 1]]


In [119]:
# Calculate PoI
P1 = np.zeros((N, num_cells*outdim))
P2 = np.zeros((N, num_cells*outdim))
for i in range(len(ub_idx)):
    for k in range(N):
        P1[k,i] = np.transpose(Phi, [1, 2, 0])[ub_idx[i][0],ub_idx[i][1], k]  # N x num_cell*outdim
        P2[k,i] = np.transpose(Phi, [1, 2, 0])[lb_idx[i][0],lb_idx[i][1], k]  # N x num_cell*outdim
P = np.reshape(P1 - P2, [N, num_cells, outdim])
PoI = np.sum(np.prod(P, axis=2), axis=1, keepdims=True)  # N x 1

In [122]:
np.transpose(Phi, [1, 2, 0])

array([[[0.  ],
        [0.  ]],

       [[0.03],
        [0.99]],

       [[0.04],
        [0.98]],

       [[0.98],
        [0.02]],

       [[0.99],
        [0.01]],

       [[1.  ],
        [1.  ]]])

In [120]:
np.reshape(P1, [N,num_cells,outdim])

array([[[0.03, 1.  ],
        [0.04, 0.99],
        [0.98, 0.98],
        [0.99, 0.02],
        [1.  , 0.01]]])

In [121]:
np.reshape(P2, [N,num_cells,outdim])

array([[[0.  , 0.  ],
        [0.03, 0.  ],
        [0.04, 0.  ],
        [0.98, 0.  ],
        [0.99, 0.  ]]])

In [123]:
# Calculate Hypervolume contribution of points Y
ub_points = np.zeros((1, num_cells*outdim))
lb_points = np.zeros((1, num_cells*outdim))
for i in range(len(ub_idx)):
    ub_points[0,i] = pf_ext[ub_idx[i][0],ub_idx[i][1]]
    lb_points[0,i] = pf_ext[lb_idx[i][0],lb_idx[i][1]]
ub_points = np.reshape(ub_points, [num_cells, outdim])
lb_points = np.reshape(lb_points, [num_cells, outdim])

In [126]:
splus_valid = np.all(np.tile(np.expand_dims(ub_points, 1), [1, N, 1]) > candidate_mean,
                            axis=2)  # num_cells x N
splus_idx = np.expand_dims(splus_valid.astype(np.float64), -1)  # num_cells x N x 1
splus_lb = np.tile(np.expand_dims(lb_points, 1), [1, N, 1])  # num_cells x N x outdim
splus_lb = np.maximum(splus_lb, candidate_mean)  # num_cells x N x outdim
splus_ub = np.tile(np.expand_dims(ub_points, 1), [1, N, 1])  # num_cells x N x outdim
splus = np.concatenate([splus_idx, splus_ub - splus_lb], axis=2)  # num_cells x N x (outdim+1)
Hv = np.transpose(np.sum(np.prod(splus, axis=2), axis=0, keepdims=True))  # N x 1


In [131]:
np.tile(np.expand_dims(lb_points, 1), [1, N, 1])

array([[[       -inf,        -inf]],

       [[-0.77380929,        -inf]],

       [[-0.77337979,        -inf]],

       [[-0.74043215,        -inf]],

       [[-0.6893123 ,        -inf]]])

In [132]:
splus_lb

array([[[-0.76821882,  0.29268916]],

       [[-0.76821882,  0.29268916]],

       [[-0.76821882,  0.29268916]],

       [[-0.74043215,  0.29268916]],

       [[-0.6893123 ,  0.29268916]]])

In [133]:
splus

array([[[ 0.        , -0.00559047,  0.37431631]],

       [[ 0.        , -0.00516097,  0.15198108]],

       [[ 1.        ,  0.02778666,  0.13196399]],

       [[ 0.        ,  0.05111985, -0.0666407 ]],

       [[ 0.        ,  0.04224849, -0.29268937]]])

In [134]:
np.prod(splus, axis=2)

array([[-0.        ],
       [-0.        ],
       [ 0.00366684],
       [-0.        ],
       [-0.        ]])

In [65]:
np.sum(np.logical_and(np.all(extended <= np.swapaxes(extended, 0, 1), axis=2),
                                  np.any(extended < np.swapaxes(extended, 0, 1), axis=2)), axis=1)

array([2, 0, 4, 0, 2, 3, 0, 0])

In [86]:
n_params = bounds.shape[0]
points = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(10, 2))
points

array([[22.98427679, 18.32308058],
       [18.5577584 , 17.79271085],
       [ 1.92835755,  7.21801334],
       [21.99461056,  8.46549208],
       [18.35027392, 18.7385134 ],
       [18.67232717, 12.16198247],
       [ 4.98785331, 24.850071  ],
       [ 0.57735326,  0.38854877],
       [15.32600135,  6.14972245],
       [12.79589151, 13.938944  ]])

In [37]:
num_output = 2
pareto = Pareto(np.empty((0, num_output)))
ref = np.array([[3,3]]) 
y = np.array([[0,2], [2,0], [1,1]])
pareto.update(y)
pf = pareto.front
ext1, ext2 = np.array([[-np.inf, ref[0][1]]]), np.array([[ref[0][0], -np.inf]])
pf_ext = np.concatenate([ext1, pareto.front, ext2], 0)

predicted_mean, predicted_var = np.array([[0,0]]), np.array([[0.01,0.01]])
normal1 = scipy.stats.norm(predicted_mean[:,0], np.sqrt(predicted_var[:,0]))
normal2 = scipy.stats.norm(predicted_mean[:,1], np.sqrt(predicted_var[:,1]))

P1 = np.zeros((1, len(pf)+1))
P2 = np.zeros((1, len(pf)+1))
for i in range(len(pf)+1):
    P1[0,i] = (pf_ext[i+1, 0] - pf_ext[i, 0]) * normal1.cdf(pf_ext[i, 0]) * exipsi(pf_ext[i, 1], pf_ext[i, 1], normal2) 
    P2[0,i] = (exipsi(pf_ext[i+1, 0], pf_ext[i+1, 0], normal1) - exipsi(pf_ext[i+1, 0], pf_ext[i, 0], normal1)) * exipsi(pf_ext[i, 1], pf_ext[i, 1], normal2)
    if i == 0:
        P1[i] = 0

 /home/cwj/venv/local/lib/python3.5/site-packages/ipykernel_launcher.py:17: RuntimeWarning:invalid value encountered in multiply


In [45]:
np.sum(P1 + P2)

3.7978845608028653

In [109]:
def EHVI(Xcand, gp_models, pareto, reference, wind_dir, outdim = 2):
    Xcand = np.atleast_2d(Xcand)
    Xcand = np.hstack((Xcand, np.ones((len(Xcand), 1)) * wind_dir))

    ext1, ext2 = np.array([[-np.inf, reference[0][1]]]), np.array([[reference[0][0], -np.inf]])
    pf_ext = np.concatenate([ext1, pareto.front, ext2], 0)

    # Predictions for candidates, concatenate columns
    preds = [m.predict(Xcand) for m in gp_models]
    candidate_mean, candidate_var = (np.concatenate(moment, 1) for moment in zip(*preds))
    candidate_var = np.maximum(candidate_var, 1e-6)  # avoid zeros
    
    normal1 = scipy.stats.norm(candidate_mean[:,0], np.sqrt(candidate_var[:,0]))
    normal2 = scipy.stats.norm(candidate_mean[:,1], np.sqrt(candidate_var[:,1]))
    
    P1 = np.zeros((points.shape[0], len(pareto.front)+1))
    P2 = np.zeros((points.shape[0], len(pareto.front)+1))

    for i in range(len(pf)+1):
        P1[:,i] = (pf_ext[i+1, 0] - pf_ext[i, 0]) * normal1.cdf(pf_ext[i, 0]) * exipsi(pf_ext[i, 1], pf_ext[i, 1], normal2) 
        P2[:,i] = (exipsi(pf_ext[i+1, 0], pf_ext[i+1, 0], normal1) - exipsi(pf_ext[i+1, 0], pf_ext[i, 0], normal1)) * exipsi(pf_ext[i, 1], pf_ext[i, 1], normal2)
        if i == 0:
            P1[:,i] = 0
            
    return -np.sum(P1 + P2, axis = 1)[:,None]

def exipsi(a, b, normal):
    return normal.std() * normal.pdf(b) + (a-normal.mean()) * normal.cdf(b)

In [110]:
n_params = bounds.shape[0]
points = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(10, n_params))

EHVI(points, gp_models, pareto, reference, wd_context[i], outdim = 2)

 /home/cwj/venv/local/lib/python3.5/site-packages/ipykernel_launcher.py:20: RuntimeWarning:invalid value encountered in multiply


array([[-4.13449953e-02],
       [-2.70182969e-02],
       [-7.18810937e-18],
       [-1.45068752e-20],
       [-3.33597523e-04],
       [-1.68288111e-12],
       [-2.91586712e-07],
       [-7.25272427e-27],
       [-1.74504667e-03],
       [-3.58715442e-17]])

In [80]:
hypervolume_poi(next_point, gp_models, pareto, reference, 2, wd_context[i])

array([[-0.000297]])

In [8]:
np.ones([1, 2])

array([[1., 1.]])

In [6]:
f = np.max(pareto.front, axis=0, keepdims=True) - np.min(pareto.front, axis=0, keepdims=True)
np.max(pareto.front, axis=0, keepdims=True) + 2 * f / pareto.front.shape[0]

array([[3.33333333, 3.33333333]])